In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# import library
import os
import datetime
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import metrics, losses, optimizers
from tensorflow.keras import backend, layers, models, callbacks, utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

from matplotlib import pyplot as plt

plt.style.use('dark_background')

In [3]:
# define plot function
def plot_loss(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, label="training loss")
    plt.plot(epochs, val_loss, label="valid loss")
    plt.title("Training & Valid Loss")
    plt.legend()
    plt.show()


def plot_acc(history):
    acc = history.history["categorical_accuracy"]
    val_acc = history.history["val_categorical_accuracy"]
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, label="training acc")
    plt.plot(epochs, val_acc, label="valid acc")
    plt.title("Training & Valid Accuracy")
    plt.legend()
    plt.show()

In [4]:
# define our working directory
img_dir = os.path.join("../subset_stanford_online_products/")
save_dir = os.path.join("models/")

In [5]:
img_size = (299, 299, 3)
batch_size = 32

lr_classifier = 1e-4
epoch_classifier = 10

lr_tuning = 1e-6
epoch_tuning = 12

In [6]:
train_datagen = image.ImageDataGenerator(rotation_range=40,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2,
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         horizontal_flip=True,
                                         fill_mode="nearest",
                                         validation_split=0.2)

# Note that the validation data should not be augmented!
test_datagen = image.ImageDataGenerator(validation_split=0.2)

In [7]:
train_generator = train_datagen.flow_from_directory(
    # This is the target directory
    img_dir,
    subset="training",
    # All images will be resized to 299 x 299
    target_size=img_size[:2],
    batch_size=batch_size,
    # Since we use categorical_crossentropy loss, we need categorical labels
    class_mode="categorical",
    shuffle=True,
    seed=1234)

validation_generator = test_datagen.flow_from_directory(
    img_dir,
    subset="validation",
    target_size=img_size[:2],
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=1234)

Found 9600 images belonging to 12 classes.
Found 2400 images belonging to 12 classes.


In [8]:
def create_model():
    conv_base = Xception(weights="imagenet",
                         include_top=False,
                         input_shape=img_size)

    conv_base.trainable = False

    input_layer = keras.Input(shape=img_size)
    x = preprocess_input(input_layer)
    x = conv_base(x, training=False)
    x1 = layers.GlobalAveragePooling2D()(x)
    x2 = layers.GlobalMaxPooling2D()(x)
    x = layers.Concatenate()([x1, x2])
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    output_layer = layers.Dense(12, activation="softmax")(x)

    model = keras.Model(inputs=input_layer, outputs=output_layer)

    return conv_base, model

In [9]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [10]:
my_callbacks = [
    callbacks.CSVLogger(os.path.join(save_dir, "log_training.csv"),
                        separator=",",
                        append=True)
]

In [11]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    conv_base, model = create_model()
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=optimizers.Adam(learning_rate=lr_classifier),
                  metrics=[metrics.CategoricalAccuracy()])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [12]:
conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [14]:
model.evaluate(validation_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 12s 165ms/step - loss: 3.3014 - categorical_accuracy: 0.0737


[3.3013506285349528, 0.07375]

In [15]:
model.load_weights(
    os.path.join("transfer_learning_classifier_preprocess_input.h5"),
    by_name=False,
    skip_mismatch=False)

In [16]:
model.evaluate(validation_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 11s 141ms/step - loss: 0.7051 - categorical_accuracy: 0.7708


[0.7051020739475886, 0.7708333]

# Fine tune from add 8

In [17]:
try:
    del (conv_base)
    del (model)
except:
    pass

In [18]:
backend.clear_session()

In [19]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    conv_base, model = create_model()
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=optimizers.Adam(learning_rate=lr_classifier),
                  metrics=[metrics.CategoricalAccuracy()])

model.load_weights(
    os.path.join("transfer_learning_classifier_preprocess_input.h5"),
    by_name=False,
    skip_mismatch=False)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [21]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'add_8':
        set_trainable = True

    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [22]:
with strategy.scope():
    model.compile(optimizer=optimizers.Adam(learning_rate=lr_tuning),
                  loss=losses.CategoricalCrossentropy(),
                  metrics=[metrics.CategoricalAccuracy()])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [23]:
history = model.fit(train_generator,
                    epochs=epoch_tuning,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=my_callbacks)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 300 steps, validate for 75 steps
Epoch 1/12
INFO:tensorflow:batch_all_reduce: 51 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 51 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
300/300 [==============================] - 143s 478ms/step - loss: 0.8266 - categorical_accuracy: 0.7347 - val_loss: 0.7001 - val_categorical_accuracy: 0.7717
Epoch 2/12
300/300 [==============================] - 134s 448ms/step - loss: 0.8412 - categorical_accuracy: 0.7290 - val_loss: 0.6964 - val_categorical_accuracy: 0.7733
Epoch 3/12
300/300 [==============================] - 135s 449ms/step - loss: 0.8252 - categorical_accuracy: 0.7320 - val_loss: 0.6947 - val_categorical_accuracy: 0.7742
Epoch 4/12
300/300 [==============================] - 134s 446ms/step - loss: 0.8147 - catego

In [24]:
model.evaluate(validation_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 11s 145ms/step - loss: 0.6838 - categorical_accuracy: 0.7800


[0.6837612624963124, 0.78]

# Fine tune from add 4

In [25]:
try:
    del (conv_base)
    del (model)
except:
    pass

In [26]:
backend.clear_session()

In [27]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    conv_base, model = create_model()
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=optimizers.Adam(learning_rate=lr_classifier),
                  metrics=[metrics.CategoricalAccuracy()])

model.load_weights(
    os.path.join("transfer_learning_classifier_preprocess_input.h5"),
    by_name=False,
    skip_mismatch=False)

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [29]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'add_4':
        set_trainable = True

    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [30]:
with strategy.scope():
    model.compile(optimizer=optimizers.Adam(learning_rate=lr_tuning),
                  loss=losses.CategoricalCrossentropy(),
                  metrics=[metrics.CategoricalAccuracy()])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [31]:
history = model.fit(train_generator,
                    epochs=epoch_tuning,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=my_callbacks)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 300 steps, validate for 75 steps
Epoch 1/12
INFO:tensorflow:batch_all_reduce: 99 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 99 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
300/300 [==============================] - 150s 498ms/step - loss: 0.8299 - categorical_accuracy: 0.7381 - val_loss: 0.6999 - val_categorical_accuracy: 0.7733
Epoch 2/12
300/300 [==============================] - 138s 461ms/step - loss: 0.8241 - categorical_accuracy: 0.7347 - val_loss: 0.6990 - val_categorical_accuracy: 0.7738
Epoch 3/12
300/300 [==============================] - 138s 461ms/step - loss: 0.8167 - categorical_accuracy: 0.7402 - val_loss: 0.6962 - val_categorical_accuracy: 0.7763
Epoch 4/12
300/300 [==============================] - 138s 461ms/step - loss: 0.8067 - catego

In [32]:
model.evaluate(validation_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 11s 144ms/step - loss: 0.6755 - categorical_accuracy: 0.7825


[0.6755375293890635, 0.7825]

# Fine tune all layer

In [33]:
try:
    del (conv_base)
    del (model)
except:
    pass

In [34]:
backend.clear_session()

In [35]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    conv_base, model = create_model()
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=optimizers.Adam(learning_rate=lr_classifier),
                  metrics=[metrics.CategoricalAccuracy()])

model.load_weights(
    os.path.join("transfer_learning_classifier_preprocess_input.h5"),
    by_name=False,
    skip_mismatch=False)

In [36]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [37]:
conv_base.trainable = True

In [38]:
with strategy.scope():
    model.compile(optimizer=optimizers.Adam(learning_rate=lr_tuning),
                  loss=losses.CategoricalCrossentropy(),
                  metrics=[metrics.CategoricalAccuracy()])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [39]:
history = model.fit(train_generator,
                    epochs=epoch_tuning,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=my_callbacks)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 300 steps, validate for 75 steps
Epoch 1/12
INFO:tensorflow:batch_all_reduce: 162 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 162 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
300/300 [==============================] - 199s 665ms/step - loss: 0.8102 - categorical_accuracy: 0.7432 - val_loss: 0.7018 - val_categorical_accuracy: 0.7721
Epoch 2/12
300/300 [==============================] - 188s 626ms/step - loss: 0.8119 - categorical_accuracy: 0.7447 - val_loss: 0.6977 - val_categorical_accuracy: 0.7746
Epoch 3/12
300/300 [==============================] - 188s 627ms/step - loss: 0.8089 - categorical_accuracy: 0.7403 - val_loss: 0.6903 - val_categorical_accuracy: 0.7771
Epoch 4/12
300/300 [==============================] - 188s 626ms/step - loss: 0.8034 - cate

In [40]:
model.evaluate(validation_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 11s 148ms/step - loss: 0.6701 - categorical_accuracy: 0.7862


[0.6701074024041493, 0.78625]